In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
import pandas as pd

#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))

path_fig = '/home/demitau/current_owncloud/output/memerr/laptop_home'
os.environ['FIG_MEMORY_ERRORS_STAB_AND_STOCH'] = path_fig

import gc

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
print('seaborn ver = ', sns.__version__)
#assert sns.__version__ == '0.12.2', sns.__version__
import matplotlib as mpl

In [ ]:
#d = '/home/demitau/current_owncloud/merr_data/travel_beh_an/ccpurebeh'
d = '/home/demitau/data_Quentin/pilot_studies/context_change_behav/'
#fnb = 'dima_test_context_change_20230309_164851'
#fnb = '2023-SE1-pilot2_context_change_20230310_161928'
fnb = '2023-SE1-017_context_change_20230425_161049'
fn = fnb + '.log'
fnp = fnb + '.param'

In [ ]:
fnf = pjoin(d,'es_trunc_row=trial.pkl.zip')

In [ ]:
from behav_proc import readParamFiles, addBasicInfo, getGeomInfo
from datetime import datetime 

fnp = fnb + '.param'
params, phase2trigger, trigger2phase, stage2pars  = readParamFiles(fnp, d)
home_position, target_coords = getGeomInfo(params)
hitr = float(params['radius_target']) + float(params['radius_cursor']) / 2

load = True
#load = True
if load:
    #dfallphs = pd.read_pickle(pjoin(d,'row=frame_allph.pkl.zip'))
    dfc_all      = pd.read_pickle(pjoin(d,'row=frame.pkl.zip'))
    dfcc_all     = pd.read_pickle(pjoin(d,'row=trial.pkl.zip'))
    dfcpc_all    = pd.read_pickle(pjoin(d,'row=pause.pkl.zip'))
    dfctc_all    = pd.read_pickle(pjoin(d,'row=target_chagne.pkl.zip'))

coln_pairs = [ ('error_lh_ang','error_unpert_lh_ang','error_lh_ang_nonhit','lh_ang'),
        ('error_area2_signed_nn_scaled_ed', None,
               'error_area2_signed_nn_scaled_ed_nonhit',
                'signed_area2_nn_scaled_ed') ] 

fnf = pjoin(d,'es_trunc_row=trial.pkl.zip')
dt = datetime.fromtimestamp(os.stat(fnf).st_mtime )
print(dt)

df_all_multi_tsz_orig = pd.read_pickle(fnf)
assert set( df_all_multi_tsz_orig['error_data_for_calc'] ) == set(  [cp[-1] for cp in coln_pairs])

subjects = dfcc_all['subject'].unique()
print(subjects)

subj = subjects[3]

In [ ]:
qsos = 'subject == @subj'
dfc = dfc_all.query(qsos)
dfcc = dfcc_all.query(qsos)
dfcpc = dfcpc_all.query(qsos)
dfctc = dfctc_all.query(qsos)
df_multi_tsz = df_all_multi_tsz.query(qsos)

# Recalc ES

In [ ]:
assert 'error' not in dfcc_all.columns
#df_all = dfcc_all.drop(labels=['index', 'level_0'],axis=1)  #.copy()
#df_all['non_hit_not_adj'] = df_all['nonhit']
dfcc_escalc = dfcc.copy()
dfcc_escalc['target_inds']     = dfcc['tgti_to_show']
dfcc_escalc['trials']          = dfcc['trial_index']
dfcc_escalc['environment'] = 1111

# for err sens calc
#df_all['error']          = df_all['error_area_signed']
##############################
subjects_ = [subj]
print(subjects_)

In [ ]:
q = 0.04
a = dfcc_escalc['error_lh_ang'].abs()
thr = a.quantile(q); print(thr)
dfcc_escalc['error_lh_ang_nonhit'] = a > thr

a = dfcc_escalc['error_area2_signed_nn_scaled_ed'].abs()
thr = a.quantile(q); print(thr)
dfcc_escalc['error_area2_signed_nn_scaled_ed_nonhit'] = a > thr

In [ ]:
dfcc_escalc['error_area2_signed_nn_scaled_ed'].abs().describe()

In [ ]:
from behav_proc import computeErrSensVersions
envs_cur = ['all']
tgt_inds_cur =[None]

block_names_cur = ['all']
pertvals_cur = [None]
gseqcs_cur = [ None ]
dists_rad_from_prevtgt_cur = [None]
dists_trial_from_prevtgt_cur = [None]
error_type = 'MPE'
#time_lockeds=['target', 'feedback', 'trial_end']
time_lockeds=['target', 'trial_end']

coln_pairs = [ ('error_area2_signed_nn_scaled_ed',
                None,
               'error_area2_signed_nn_scaled_ed_nonhit',
                'signed_area2_nn_scaled_ed') ]
coln_pairs += [('error_lh_ang','error_unpert_lh_ang','error_lh_ang_nonhit','lh_ang')]

from base2 import areaBetweenTraj, calcNormCoefSectorArea
norm_coef = calcNormCoefSectorArea(params)
def trajPair2corr(dftraj1, dftraj2):
    return areaBetweenTraj(dftraj1, dftraj2, home_position,                                                                                                  
                target_coords, params ) * norm_coef

dfouts = []
import sys, traceback
try:
    for coln_error,coln_correction_calc,coln_nh, calc_name  in coln_pairs:
        for time_locked in time_lockeds:
            df_all_multi_tsz, ndf2vn = computeErrSensVersions(dfcc_escalc, envs_cur,block_names_cur,
                pertvals_cur,gseqcs_cur,tgt_inds_cur,
                dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,
                subj_list = subjects_, error_type=error_type,
                trial_shift_sizes = [1],
                addvars=[], target_info_type = 'locs',                                        
                computation_ver = 'computeErrSens3', 
                coln_nh = coln_nh,
                time_locked = time_locked,
                coln_nh_out = coln_nh + '_shifted',
                coln_error =coln_error,                                                                              
                coln_correction_calc = coln_correction_calc,                
                trajPair2corr = trajPair2corr,
                df_fulltraj = dfc,
                verbose=1,)
            df_all_multi_tsz['error_data_for_calc'] = calc_name
            dfouts += [df_all_multi_tsz]
    df_all_multi_tsz = pd.concat(dfouts).reset_index()
    
except Exception as e:
    print('EXC',e)
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    ei = exc_info[2]

    #display(ei.tb_frame)
    psf = ei
    
    lfprev = None
    lfs = []
    lf = None
    while psf.tb_frame is not None:
        stackframe = psf.tb_frame
        display(stackframe)
        lfprev = lf
        lf = stackframe.f_locals
        lfs += [lf]
        psf = psf.tb_next    
        if psf is None:
            break

    print('EXC',e)

if coln_correction_calc is not None:
    df_all_multi_tsz['coln_corrc'] = df_all_multi_tsz[coln_correction_calc]
df_all_multi_tsz['coln_error'] = df_all_multi_tsz[coln_error]

bigEStis = df_all_multi_tsz.query('err_sens > 10')['trial_index'].to_numpy()
display(bigEStis)
bigEStis_neg = df_all_multi_tsz.query('err_sens < -10')['trial_index'].to_numpy()
#display(bigEStis_neg)

# Double traj vert aligned

In [ ]:
def trajPair2corr_plot(dftraj1, dftraj2, ax):
    return areaBetweenTraj(dftraj1, dftraj2, home_position,                                                                                                  
                target_coords, params, ax=ax ) * norm_coef

In [ ]:
# plot double traj

#ax = plt.gca()
correction = []


show_dist_guides = False
ww = 5; hh = 5

#nshow = 25\
nshow = 25
rng = slice(1,1+nshow)
#ti_ = 103; 
ti_ = 450; 
rng = slice(ti_ - (nshow//2), ti_ + (nshow//2) )

#L = min(nshow, len(trial_inds_glob1) -2 )
L = min(nshow, len(trial_inds_glob1)  )
nc = int(np.ceil(np.sqrt(L) ) ); nr = int( np.ceil( L / nc ) )
#nr = N; nc = 2

fig,axs = plt.subplots(nr,nc, figsize=(nc*ww, nr * hh))
axsf = axs.flatten()

axi = 0
#for trial_inds_glob1, valid_inds1, trial_inds_glob2
#for gti1,gti2 in list(zip(trial_inds_glob1, \
#                          trial_inds_glob2))[rng]:
for gti1,gti2 in list(zip(trial_inds_glob1, \
                          trial_inds_glob2))[:-1]:
    
    ax = axsf[axi]
    
    dftraj1 = dfc.query( 'trial_index == @gti1' )
    dftraj2 = dfc.query( 'trial_index == @gti2' )

    dftraj1 = dftraj1.loc[dftraj1.index[1: ] ]
    dftraj2 = dftraj2.loc[dftraj2.index[1: ] ]

    ab = trajPair2corr_plot(dftraj1, dftraj2, ax=ax  )

    correction_cur = ab
    correction += [correction_cur]
    #es = correction_cur/errors0[gti1]
    #s = ''
    #if es > 10:
    #    s = '*'
    #print(s, gti1,gti2,correction_cur,
    #      errors0[gti1], es )
    axi += 1

plt.tight_layout()
plt.savefig(pjoin(path_fig, f'doubletraj_2D_{subj}_{rng.start,rng.stop}.pdf') )

# when first is on the right of second, then we have positive
# but to be similar to Romain's version we would like to have neg
# or not?

# Single traj full info

In [ ]:
rng = np.arange(18,27)

In [ ]:
grp_perti = dfc.groupby(['trial_index'])

In [ ]:
# maybe try to exclude blocks where at least one trial was bad?

In [ ]:
df_all_multi_tsz['time_locked'].unique()

In [ ]:
cols = ['subject', 'trial_index', 'error_data_for_calc',
       'trial_shift_size', 'trial_group_col_calc', 'time_locked']
df_all_multi_tsz.duplicated(cols).any()

In [ ]:
df_all_multi_tsz['trial_group_calc'].unique()

In [ ]:
edfc = 'lh_ang'
tl = 'target'
#tl = 'trial_end'
qs_es = (f'subject == "{subj}"' 
          ' and error_data_for_calc == @edfc'
          ' and trial_shift_size == 1 and trial_group_col_calc == "trials"'
          ' and time_locked == @tl'
         )  

df_all_multi_tsz.query(qs_es)

In [ ]:
coln_error, coln_correction_calc, _, _,  = 'error_lh_ang','error_unpert_lh_ang','error_lh_ang_nonhit','lh_ang'

In [ ]:
colscols = [['trialwb','perturbation','trial_type'],
            ['prev_trial_type'],
            ['error_endpoint_ang', 'error_lh_ang'],            
            ['time_mvt', 'traj_length'],            
            [coln_correction_calc], [coln_error],
            [ 'correction', 'prev_error'],
           ['err_sens','prev_err_sens', 'nonhit']]

rngs = f'{rng[0]}-{rng[-1]}'; print(rngs)

# Double traj not ta

In [ ]:
path_fig = '/home/demitau/current_owncloud/output/memerr/laptop_home'

In [ ]:
from behav_proc import plotTraj, plotTraj2, row2multierr
#from config2 import path_fig
#rng = range(600,603)
#rng = range(596,625)
#rng = range(8,10)
#rng = range(50,60)
#rng = range(55,57)

show_dist_guides = False
ww = 7; hh =10
nc = int(np.ceil(np.sqrt(len(rng)) ) ); nr = int( np.ceil( len(rng) / nc ) )
#nr = N; nc = 2

fig,axs = plt.subplots(nr,nc, figsize=(nc*ww, nr * hh))
axsf = axs.flatten()

axi2d = {}
for axi,ti in enumerate(rng):
    ax = axsf[axi]    
    if ti not in grp_perti.groups:
        if ti in pause_inds:            
            print(f'Skipping {ti} because pause')
            ax.set_title('PAUSE')
        else:
            print(f'Skipping {ti} because ?')
        continue
        
    df0 = dfcc.query(f'subject == "{subj}" and trial_index == {ti}')
    row = df0.to_dict('records')[0]
    
    ev = row['error_endpoint_ang']
    tiwb = row['trialwb']
    
    dfes = df_all_multi_tsz.query(qs_es + f' and trial_index == {ti}')
    assert len(dfes) == 1, len(dfes)
    row2 = dfes.to_dict('records')[0]
    row.update(row2)
    
    
    idx = grp_perti.groups[ti]
    dfcurtr = dfc.loc[idx[1:]]  # zeroth is equal to zero
    import sys, traceback
    try:
#         plotTraj2(ax, dfcurtr, home_position, target_coords_homec, params,
#                 force_entire_traj=0, calc_area = 1, addinfo=row,
#                 titlecols = colscols, xlim=(-160,160))
        
        r = row2multierr(row, dfc, grp_perti, home_position,             
            target_coords, params, ax=ax,
            revert_pert = False, xlim=(-160,160),
            titlecols = colscols, addinfo=row, calc_area = 0)
        z = zip( ['err', 'error_lh_ang', 'error_unpert_lh_ang', 'start_dist',
                      'traja', 'trajoa', 'td',
                'length',  'enddist'], r)
        dd = dict(z)
    
    except Exception as e:
        print('EXC',e)
        exc_info = sys.exc_info()
        exc = traceback.TracebackException(*exc_info, capture_locals=True)
        ei = exc_info[2]

        #display(ei.tb_frame)
        psf = ei

        lfprev = None
        lfs = []
        lf = None
        while psf.tb_frame is not None:
            stackframe = psf.tb_frame
            display(stackframe)
            lfprev = lf
            lf = stackframe.f_locals
            lfs += [lf]
            psf = psf.tb_next    
            if psf is None:
                break
        raise e
        break
    
    from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
    #ax.xaxis.set_major_locator(MultipleLocator(50))
    #ax.xaxis.set_major_locator(MultipleLocator(50))
    ax.set_yticks(np.arange(-50,400,50))
    #ax.xaxis.set_minor_locator(AutoMinorLocator(25))
    ax.grid(which='major', color='#CCCCCC', linestyle='--')
    #ax.grid(which='minor', color='#CCCCCC', linestyle=':')
    axi2d[axi] = {'df0':df0, 'row':row, 'dfcurtr':dfcurtr}
    
    print(f'error = {ev}, tiwb = {tiwb}')
    
plt.tight_layout()
plt.savefig(pjoin(path_fig, f'traj_2D_{row["subject"]}_{rngs}.pdf') )
plt.close()

In [ ]:
dftmp['perturbation']

In [ ]:
def pptau(d):
    for k,v in d.items():
        if not isinstance(v,np.ndarray):
            if isinstance(v,float):
                print(f'{k} == {v:.2f}')
            else:
                print(f'{k} == {v}')

In [ ]:
rng = np.arange(25,45)
#rng = np.arange(25,27)

In [ ]:
ww = 7; hh =10
nc = int(np.ceil(np.sqrt(len(rng)) ) ); nr = int( np.ceil( len(rng) / nc ) )
#nr = N; nc = 2

fig,axs = plt.subplots(nr,nc, figsize=(nc*ww, nr * hh))
axsf = axs.flatten()

axi2d = {}
for axi,ti in enumerate(rng):
    if ti == 0:
        continue
    ax = axsf[axi]    

    tind1 = ti
    tind2 = tind1 -1
    from behav_proc import analyzeTraj

    tmp = {}
    tmpo = {}
    for tind in [tind1, tind2]:
        print(tind)
        dftmp = dfc.query('trial_index == @tind')
        fbXY = dftmp[['feedbackX','feedbackY']]
        tgt = dftmp[['target_coordX','target_coordY']].iloc[0].values
        rd = analyzeTraj(fbXY.values, tgt, home_position, 
                    params['radius_home'])
        tmp[tind] = rd
    
        ofbXY = dftmp[['unpert_feedbackX','unpert_feedbackY']]
        rd = analyzeTraj(ofbXY.values, tgt, home_position, 
                    params['radius_home'], indlh=rd['indlh'])
        tmpo[tind] = rd


    rt = params['radius_target']
    rh = params['radius_home']
    pp_per_tind = [dict(ls='-', c='b', marker='o', markersize=2), 
                   dict(ls=':', c='b', marker='o', markersize=2)]
    ppo_per_tind = [dict(ls='-', c='orange', marker='o', markersize=2), 
                   dict(ls=':', c='orange', marker='o', markersize=2)]
    for tind,pp,ppo in zip([tind1, tind2],pp_per_tind,ppo_per_tind):
        tgtcur = tmp[tind]['tgtcur']
        rd = tmp[tind]
        print('   ' ,tind)        
        crc = plt.Circle(tgtcur, rt, color='blue', lw=2, fill=False,
                         alpha=0.6, ls=pp['ls'])
        ax.add_patch(crc)
        ax.plot( *rd['XYhc'].T, **pp)

        c = (1 / np.pi) * 180

        indlh = rd['indlh']
        XYlh = rd['XYhc'][indlh]
        ang_lh = rd['ang_lh']
        print('0pt={}, lhpt={}'.format(rd['pt0'], XYlh ))

        print('lh={:5.6f}, tgt={:5.2f}, err={:5.2f} '.format(ang_lh*c, rd['ang_tgt'] * c, 
              rd['error_lh_ang'] * c) )
        
        ax.scatter( *XYlh , alpha=1,
            c='k', s= 15, 
            label=f'{tind} ang_lh_0pt = {ang_lh*c:.1f}')        
        
        rdo = tmpo[tind]
        ax.plot( *rdo['XYhc'].T, **ppo)                
        
        
    ax.scatter( 0,0, alpha=0.8,
            c='k', s= 25 )
    crc = plt.Circle((0, 0), rh, color='r', lw=2, fill=False,
                     alpha=0.3)
    ax.add_patch(crc)
    ax.legend()
    
    s = '\n'
    r = df_all_multi_tsz.query(qs_es + ' and trial_index == @tind').to_dict('records')[0]
    for cols_ in colscols:
        for col in cols_:
            if col is None:
                continue
            s += f'{col}='            
            colv = r[col]
            if isinstance(colv,float):
                s += f'{colv:.2f}'
            else:
                s += f'{colv}'
            s+='; '
        s += '\n'

    ax.set_title(f'ti={tind1}; pert={r["perturbation"]}; ' + s)
    
plt.tight_layout()
plt.savefig(pjoin(path_fig, f'trajdouble_2D_{row["subject"]}_{rngs}.pdf') )
plt.close()


In [ ]:
dfes

In [ ]:
subjects

In [ ]:
qs_es

In [ ]:
#dfes.query('block_ind >= 0')

In [ ]:
#dfes

In [ ]:
sns.violinplot?

In [ ]:
fg  = sns.catplot(data=dfes_goog, kind='violin',  y='err_sens', x='trial_type')
for ax in fg.axes.flatten():
    ax.axhline(0, ls=':', color='red')
    ax.set_ylim(-2,2)

In [ ]:
# 	T	dof	alternative	p-val	CI95%	cohen-d	BF10	power
# T-test	3.128035	467	greater	0.000935	[0.07, inf]	0.144593	12.775	0.930382

In [ ]:
pg.ttest?

In [ ]:
import researchpy as rpy

In [ ]:
block_starts

In [ ]:
from behav_proc import markBlockStarts

In [ ]:
dfincsp = aggRows(df, 'time', 'max', grp, coltake = 'corresp')


In [ ]:
pause_inds = dfcpc.groupby(['trial_index']).max('time').index.values #['trial_index']#.values

In [ ]:
ECs = dfes.query('trial_type == "error_clamp"')['trial_index']

In [ ]:
truncateDf?

In [ ]:
spectrials

In [ ]:
qs_notspec_not_afterspec

# Trunc effects

In [ ]:
# dependence on truncation params
from behav_proc import truncateDf
#truncateDf?

#dfes_goog_noq = truncateDf( dfes, 'err_sens', q = None, infnan_handling='discard' )

import pingouin as pg
numtrain = params['num_training']
from behav_proc import qs_notspec, qs_notspec_not_afterspec,  spectrials
print( qs_notspec )

pds = [dict(q=0.05),dict(q=0.02), dict(q=None),
      dict(q=0.05, trunc_hi = 0),dict(q=0.02, trunc_hi =0), dict(q=None, trunc_hi =0),
      dict(q=0.05, trunc_low = 0),dict(q=0.02, trunc_low =0), dict(q=None, trunc_low =0),
      dict(q=0.05, trunc_low = 0, abs=True),dict(q=0.02, trunc_low =0, abs=True), dict(q=None, trunc_low =0, abs=True)]
rs = []
for truncapd in pds:
    print('   truncapd=',truncapd)
    dfes_goog = truncateDf( dfes, 'err_sens', **truncapd, infnan_handling='discard',verbose=1 )
    r = pg.ttest(dfes_goog.query(qs_notspec)['err_sens'], 0 , alternative='greater')
    r['truncaprs'] = repr(truncapd)
    rs += [r]
    #display(pg.ttest(dfes_goog.query(qs_notspec)['err_sens'], 0 , alternative='less'))
pd.concat(rs)

In [ ]:
dfes_goog = truncateDf( dfes, 'err_sens', **truncapd, infnan_handling='discard',verbose=1 )

In [ ]:
grp_perti.apply?

In [ ]:
#dftmp.abs?

In [ ]:
import pandas as pd

In [ ]:
dfes = df_all_multi_tsz.query(qs_es)
timax = 90
fg = sns.lineplot(data=dfes.query('trial_index < @timax and trial_index >= 11'), 
                  x='trial_index', y='err_sens')
#for ax in fg.axes.flatten():
ax = fg.axes
ax.set_ylim(-2,2)
ax.axhline(0,ls=':',color='red')

block_starts = markBlockStarts(dfes, ax, dict(ymin = -1, ymax = 1, color='grey', ls=':') )

ax.vlines(pause_inds, -1,1, color='yellow',lw=2)
ax.vlines(ECs, 0.5,1, color='green',lw=2, ls='--')

ax.set_xlim(11, timax)